In [0]:
from pyspark.sql.functions import *

In [0]:
tran_silver_df = spark.read.table("retail_analytics.bronze.transactions")

In [0]:
tran_silver_df = (
    spark.read.table("retail_analytics.bronze.transactions")
    .dropDuplicates(["Invoice_ID", "Line"])
    .withColumn("Size", trim(col("Size")))
    .withColumn("Color", trim(col("Color")))
    .withColumn("Currency", trim(col("Currency")))
    .withColumn("Currency_Symbol", trim(col("Currency_Symbol")))
    .withColumn("SKU", trim(col("SKU")))
    .withColumn("Transaction_Type", trim(col("Transaction_Type")))
    .withColumn("Payment_Method", trim(col("Payment_Method")))
    .withColumn(
        "Discount",
        when(col("Discount").isNull(), 0.0).otherwise(col("Discount"))
    )
    .withColumn("invoice_date", to_date(col("Date")))
    .fillna({
        "Size": "Not available",
        "Color": "Not available",
        "Currency": "Not available",
        "Currency_Symbol": "Not available",
        "SKU": "Not available",
        "Transaction_Type": "Not available",
        "Payment_Method": "Not available"
    })
)

In [0]:
(tran_silver_df.write
 .format("delta")
 .mode("overwrite")
 .saveAsTable("retail_analytics.silver.transactions")
)

In [0]:
spark.read.table("retail_analytics.silver.transactions").limit(5).display()